In [1]:
from sklearn.datasets import load_iris

# Завантаження даних Iris
iris = load_iris()

# Для ознайомлення можна подивитись, які ключі містяться у завантаженому словнику:
print("Ключі у завантаженому наборі даних:", iris.keys())

Ключі у завантаженому наборі даних: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [7]:
from sklearn.model_selection import train_test_split

# Розподіл даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(
    iris.data,       # ознаки
    iris.target,     # мітки класів
    test_size=0.3,   # 30% даних для тестування
    random_state=42, # для відтворюваності результатів
    stratify=iris.target  # зберігаємо пропорції класів
)

# Виведемо розміри отриманих вибірок
print("Розмір навчальної вибірки:", X_train.shape)
print("Розмір тестової вибірки:", X_test.shape)

Розмір навчальної вибірки: (105, 4)
Розмір тестової вибірки: (45, 4)


In [8]:
import numpy as np

# Отримуємо унікальні класи з міток навчальної вибірки
unique_classes = np.unique(y_train)

# Створюємо словник, де ключами будуть класи, а значенням — відповідні зразки ознак
X_train_by_class = {}
for c in unique_classes:
    X_train_by_class[c] = X_train[y_train == c]
    print(f"Клас {c}: {X_train_by_class[c].shape[0]} зразків")

# Приклад: виведемо перші 5 зразків для класу 0
print("Перші 5 зразків для класу 0:\n", X_train_by_class[0][:5])

Клас 0: 35 зразків
Клас 1: 35 зразків
Клас 2: 35 зразків
Перші 5 зразків для класу 0:
 [[5.1 3.8 1.5 0.3]
 [4.9 3.1 1.5 0.1]
 [5.1 3.8 1.6 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.  1.4 0.2]]


In [9]:
# Створюємо словник для збереження коваріаційних матриць кожного класу
cov_matrices = {}

for c in unique_classes:
    # Отримуємо дані для поточного класу
    X_c = X_train_by_class[c]

    # Обчислюємо коваріаційну матрицю
    cov_matrix = np.cov(X_c, rowvar=False)

    # Зберігаємо матрицю у словнику
    cov_matrices[c] = cov_matrix

    # Виводимо отриману коваріаційну матрицю для поточного класу
    print(f"Коваріаційна матриця для класу {c}:\n{cov_matrix}\n")

Коваріаційна матриця для класу 0:
[[0.10633613 0.10794958 0.00894958 0.01282353]
 [0.10794958 0.17902521 0.01567227 0.01247059]
 [0.00894958 0.01567227 0.02361345 0.00382353]
 [0.01282353 0.01247059 0.00382353 0.00952941]]

Коваріаційна матриця для класу 1:
[[0.24786555 0.06989916 0.1722605  0.04810084]
 [0.06989916 0.08810084 0.06115126 0.03148739]
 [0.1722605  0.06115126 0.21769748 0.06584874]
 [0.04810084 0.03148739 0.06584874 0.03668908]]

Коваріаційна матриця для класу 2:
[[0.43734454 0.10632773 0.33584874 0.0347395 ]
 [0.10632773 0.12080672 0.09148739 0.04380672]
 [0.33584874 0.09148739 0.33221849 0.0492521 ]
 [0.0347395  0.04380672 0.0492521  0.0657479 ]]



In [10]:
# Створюємо словник для збереження обернених коваріаційних матриць для кожного класу
inv_cov_matrices = {}

for c in unique_classes:
    try:
        # Спробуємо обчислити обернену матрицю
        inv_cov_matrix = np.linalg.inv(cov_matrices[c])
    except np.linalg.LinAlgError:
        # Якщо коваріаційна матриця вироджена, використовуємо псевдообернену матрицю
        inv_cov_matrix = np.linalg.pinv(cov_matrices[c])
    inv_cov_matrices[c] = inv_cov_matrix

    print(f"Обернена коваріаційна матриця для класу {c}:\n{inv_cov_matrix}\n")

Обернена коваріаційна матриця для класу 0:
[[ 26.4725808  -15.00973871   2.6913364  -17.06107947]
 [-15.00973871  14.85756612  -4.5928917    2.5978826 ]
 [  2.6913364   -4.5928917   47.04680526 -16.48802863]
 [-17.06107947   2.5978826  -16.48802863 131.11287701]]

Обернена коваріаційна матриця для класу 1:
[[  9.93551105  -4.01797119  -8.39205106   5.48433003]
 [ -4.01797119  18.05565052   2.62794165 -14.94461892]
 [ -8.39205106   2.62794165  17.17293632 -22.07465095]
 [  5.48433003 -14.94461892 -22.07465095  72.51079863]]

Обернена коваріаційна матриця для класу 2:
[[ 11.35076597  -3.07021976 -11.29818332   4.51171287]
 [ -3.07021976  13.3362602    0.57148332  -7.69160272]
 [-11.29818332   0.57148332  15.125557    -5.74173597]
 [  4.51171287  -7.69160272  -5.74173597  22.25168948]]



In [11]:
# Обчислення апріорних ймовірностей для кожного класу
priors = {}
total_samples = len(y_train)

for c in unique_classes:
    count_c = np.sum(y_train == c)
    priors[c] = count_c / total_samples
    print(f"Апріорна ймовірність для класу {c}: {priors[c]:.4f}")

Апріорна ймовірність для класу 0: 0.3333
Апріорна ймовірність для класу 1: 0.3333
Апріорна ймовірність для класу 2: 0.3333


In [12]:
# Обчислення середніх векторів для кожного класу
means = {}
for c in unique_classes:
    means[c] = np.mean(X_train_by_class[c], axis=0)
    print(f"Середній вектор для класу {c}:\n{means[c]}\n")

Середній вектор для класу 0:
[4.98857143 3.42571429 1.48571429 0.24      ]

Середній вектор для класу 1:
[5.94857143 2.73142857 4.23714286 1.30857143]

Середній вектор для класу 2:
[6.68285714 3.00857143 5.63142857 2.06857143]



In [13]:
def compute_discriminant(x, means, inv_cov_matrices, cov_matrices, priors):
    """
    Обчислює значення дискримінантної функції для вектора ознак x для кожного класу.

    Параметри:
    - x: вектор ознак (1D numpy масив)
    - means: словник середніх векторів для кожного класу
    - inv_cov_matrices: словник обернених коваріаційних матриць для кожного класу
    - cov_matrices: словник коваріаційних матриць для кожного класу
    - priors: словник апріорних ймовірностей для кожного класу

    Повертає:
    - discriminants: словник, де ключ — клас, а значення — значення дискримінантної функції для x
    """
    discriminants = {}
    for c in unique_classes:
        # Обчислюємо різницю між вектором ознак і середнім вектором класу c
        diff = x - means[c]

        # Обчислюємо квадратичну форму: (x - μ)ᵀ * Σ⁻¹ * (x - μ)
        quadratic_term = -0.5 * diff.T @ inv_cov_matrices[c] @ diff

        # Обчислюємо логарифм визначника коваріаційної матриці
        # Використовуємо np.linalg.slogdet для більшої числової стабільності
        sign, logdet = np.linalg.slogdet(cov_matrices[c])
        determinant_term = -0.5 * logdet

        # Обчислюємо логарифм апріорної ймовірності
        log_prior = np.log(priors[c])

        # Підсумовуємо всі частини
        discriminants[c] = quadratic_term + determinant_term + log_prior
    return discriminants

In [17]:
def predict_qda(X, means, cov_matrices, inv_cov_matrices, priors):
    """
    Обчислює значення дискримінантних функцій та імовірності приналежності кожному класу для всіх зразків тестових даних.

    Параметри:
    - X: матриця ознак тестових зразків (numpy array розміру [n_samples, n_features])
    - means: словник середніх векторів для кожного класу
    - cov_matrices: словник коваріаційних матриць для кожного класу
    - inv_cov_matrices: словник обернених коваріаційних матриць для кожного класу
    - priors: словник апріорних ймовірностей для кожного класу

    Повертає:
    - discriminant_values: numpy масив розміру (n_samples, n_classes) зі значеннями дискримінантних функцій
    - probabilities: numpy масив розміру (n_samples, n_classes) з імовірностями приналежності до кожного класу
    - predictions: numpy масив з прогнозованими класами (для кожного зразка вибирається клас з максимальним значенням дискримінантної функції)
    """
    n_samples = X.shape[0]
    n_classes = len(means)  # або len(unique_classes), якщо він уже визначений глобально

    # Ініціалізуємо масиви для результатів
    discriminant_values = np.zeros((n_samples, n_classes))
    probabilities = np.zeros((n_samples, n_classes))
    predictions = np.zeros(n_samples, dtype=int)

    # Проходимо по кожному зразку тестових даних
    for i, x in enumerate(X):
        # Обчислюємо дискримінантні функції для поточного зразка
        disc = compute_discriminant(x, means, inv_cov_matrices, cov_matrices, priors)
        # Отримуємо значення в тому ж порядку, що й класи (наприклад, 0, 1, 2)
        disc_values = np.array([disc[c] for c in sorted(disc.keys())])
        discriminant_values[i] = disc_values

        # Перетворюємо логарифмічні значення у ймовірності через softmax
        exp_values = np.exp(disc_values)
        sum_exp = np.sum(exp_values)
        probabilities[i] = exp_values / sum_exp

        # Прогнозуємо клас за максимальним значенням дискримінантної функції
        predictions[i] = np.argmax(disc_values)

    return discriminant_values, probabilities, predictions


In [18]:
# Після отримання прогнозів за допомогою нашої функції predict_qda:
disc_vals, probs, preds = predict_qda(X_test, means, cov_matrices, inv_cov_matrices, priors)

# Виведемо для прикладу перші 10 прогнозів та відповідні реальні значення
print("Прогнозовані значення (custom QDA):", preds[:10])
print("Реальні значення:", y_test[:10])

# Для оцінки продуктивності можна обчислити точність та інші метрики.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Обчислюємо точність (accuracy)
accuracy = accuracy_score(y_test, preds)
print("Точність моделі (custom QDA): {:.2f}%".format(accuracy * 100))

# Обчислюємо матрицю помилок (confusion matrix)
conf_matrix = confusion_matrix(y_test, preds)
print("Матриця коваріації:")
print(conf_matrix)

# Обчислюємо звіт класифікації (classification report) для більш детальної оцінки
report = classification_report(y_test, preds)
print("Звіт класифікації:")
print(report)

Прогнозовані значення (custom QDA): [2 1 1 1 2 2 1 1 0 2]
Реальні значення: [2 1 2 1 2 2 1 1 0 2]
Точність моделі (custom QDA): 97.78%
Матриця плутанини:
[[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]
Звіт класифікації:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.94      1.00      0.97        15
           2       1.00      0.93      0.97        15

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [19]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Створюємо об'єкт моделі QDA
qda_model = QuadraticDiscriminantAnalysis()

# Навчаємо модель на тренувальних даних
qda_model.fit(X_train, y_train)

# Виконуємо прогнозування для тестових даних
sklearn_preds = qda_model.predict(X_test)
sklearn_probs = qda_model.predict_proba(X_test)

# Для прикладу виведемо прогнозовані значення та відповідні ймовірності для перших 10 зразків
print("Sklearn QDA - Прогнозовані значення для перших 10 зразків:", sklearn_preds[:10])
print("\nSklearn QDA - Ймовірності для перших 10 зразків:\n", sklearn_probs[:10])

# Оцінка точності моделі
accuracy_sklearn = accuracy_score(y_test, sklearn_preds)
print("\nТочність моделі (sklearn QDA): {:.2f}%".format(accuracy_sklearn * 100))

# Додатково можна вивести матрицю помилок та звіт класифікації
print("\nМатриця помилок (sklearn QDA):")
print(confusion_matrix(y_test, sklearn_preds))

print("\nЗвіт помилок (sklearn QDA):")
print(classification_report(y_test, sklearn_preds))

Sklearn QDA - Прогнозовані значення для перших 10 зразків: [2 1 1 1 2 2 1 1 0 2]

Sklearn QDA - Ймовірності для перших 10 зразків:
 [[4.50882393e-281 2.33265931e-004 9.99766734e-001]
 [5.80070002e-126 9.92322373e-001 7.67762740e-003]
 [1.26093381e-158 7.67923611e-001 2.32076389e-001]
 [1.92007252e-129 9.91317389e-001 8.68261120e-003]
 [7.01546976e-154 1.69976180e-001 8.30023820e-001]
 [7.13283390e-245 1.03400009e-008 9.99999990e-001]
 [5.47612577e-088 9.97991033e-001 2.00896736e-003]
 [1.52596086e-075 9.99976960e-001 2.30402717e-005]
 [1.00000000e+000 3.54030833e-018 2.18568869e-036]
 [2.27377265e-204 2.57429718e-007 9.99999743e-001]]

Точність моделі (sklearn QDA): 97.78%

Матриця плутанини (sklearn QDA):
[[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]

Звіт класифікації (sklearn QDA):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.94      1.00      0.97        15
           2       1.00      0.93      0.97       

### Висновок

1.	Висока точність:
Точність 97.78% свідчить про те, що реалізація правильно класифікує майже всі зразки тестових даних.
2.	Матриця помилок:
Матриця показує, що для класів 0 та 1 всі зразки класифікуються правильно, а для класу 2 лише один зразок класифіковано невірно. Це відповідає очікуваній поведінці моделі.
3.	Звіт класифікації:
Всі метрики (precision, recall, f1-score) знаходяться на дуже високому рівні для кожного класу. Це підтверджує, що модель працює коректно.
4.	Порівняння зі sklearn:
Оскільки результати custom QDA реалізації і моделі QuadraticDiscriminantAnalysis() зі sklearn ідентичні, можна зробити висновок, що алгоритм реалізовано правильно.